https://github.com/begraiPB/Data255DeepLearning/blob/main/HW12_LangChain%20and%20RAG.ipynb

Go through the demo file and run it in your local for hands-on:

Part A: Build a code understanding model. Upload your own custom code files to the model and ask questions based on the code file as context.

Part B: Write a chatbot prompt to iteratively create a sequence of chats on one particular custom data.

1. The chatbot should be able to answer the questions based on the text data or multiple documents.

2. The chatbot should save the conversation in the memory.

2. Summarize the chats at the end of the conversation.

## Part A: Build a code understanding model. Upload your own custom code files to the model and ask questions based on the code file as context.

In [ ]:
!pip install -qU \
    langchain==0.0.354 \
    openai==1.6.1 \
    datasets==2.10.1 \
    pinecone-client==3.1.0 \
    tiktoken==0.5.2

In [ ]:
import os
import openai

### Building a Chatbot (no RAG)

We will be relying heavily on the LangChain library to bring together the different components needed for our chatbot. To begin, we'll create a simple chatbot without any retrieval augmentation. We do this by initializing a `ChatOpenAI` object. For this we do need an [OpenAI API key](https://platform.openai.com/account/api-keys).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [129]:
!pip install --quiet python-dotenv
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 26.4 MB/s eta 0:00:00


In [70]:
from dotenv import load_dotenv, find_dotenv
load_dotenv('/content/drive/MyDrive/Data 255 HW/Assignment 12/.env')

True

In [71]:
import os
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
load_dotenv('/content/drive/MyDrive/Data 255 HW/Assignment 12/.env')

True

In [47]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Part A: Build a code understanding model. Upload your own custom code files to the model and ask questions based on the code file as context.

In [48]:
chat = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model=llm_model
)

In [49]:
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [179]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant who helps understand code.")
]

In [180]:
res = chat(messages)
# add latest AI response to messages
messages.append(res)
print(res.content)

Great! How can I assist you today?


In [155]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

In [156]:
code_file_path = "/content/drive/MyDrive/Data 255 HW/Assignment 12/pari_ddpm_conditional_test_2.py"
loader = TextLoader(code_file_path)
code_snippets = loader.load_and_split()

In [157]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(code_snippets)

In [158]:
texts[0]

Document(page_content='# -*- coding: utf-8 -*-\n"""pari_ddpm_conditional_test_2.ipynb\n\nAutomatically generated by Colab.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/1Q1u7dXf9i9FhGD6rvTnqLOG5kryefD33\n"""\n\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\n\n\nclass EMA:\n    def __init__(self, beta):\n        super().__init__()\n        self.beta = beta\n        self.step = 0\n\n    def update_model_average(self, ma_model, current_model):\n        for current_params, ma_params in zip(current_model.parameters(), ma_model.parameters()):\n            old_weight, up_weight = ma_params.data, current_params.data\n            ma_params.data = self.update_average(old_weight, up_weight)\n\n    def update_average(self, old, new):\n        if old is None:\n            return new\n        return old * self.beta + (1 - self.beta) * new', metadata={'source': '/content/drive/MyDrive/Data 255 HW/Assignment 12/pari_ddpm_conditional_test_2.py'})

In [159]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)

In [160]:
vectorstore.similarity_search("Can you tell me about the code in this python file?", k=3)

[Document(page_content='def launch():\n    import argparse\n    parser = argparse.ArgumentParser()\n    run_name = "DDPM_conditional"\n    epochs = 1\n    batch_size = 4\n    image_size = 64\n    num_classes = 10\n    dataset_path = "/content/drive/MyDrive/cifar-64/train"\n    device = "cuda"\n    lr = 3e-4\n    save_path = "/content/drive/MyDrive/ddpm_conditional"\n    train(run_name, device, num_classes, lr, image_size, batch_size, dataset_path, epochs, save_path)\n\nlaunch()', metadata={'source': '/content/drive/MyDrive/Data 255 HW/Assignment 12/pari_ddpm_conditional_test_2.py'}),
 Document(page_content='def forward(self, x, t, y):\n        t = t.unsqueeze(-1).type(torch.float)\n        t = self.pos_encoding(t, self.time_dim)\n\n        if y is not None:\n            t += self.label_emb(y)\n\n        x1 = self.inc(x)\n        x2 = self.down1(x1, t)\n        x2 = self.sa1(x2)\n        x3 = self.down2(x2, t)\n        x3 = self.sa2(x3)\n        x4 = self.down3(x3, t)\n        x4 = self

In [177]:
def converse_using_source_knowledge(query):
  results = vectorstore.similarity_search(query, k=3)
  source_knowledge = "\n".join([x.page_content for x in results])
  augmented_prompt = f"""Using the contexts below, answer the query.

  Contexts:
  {source_knowledge}

  Query: {query}"""

  # create a new user prompt
  prompt = HumanMessage(
      content=augmented_prompt
  )
  # add to messages
  messages.append(prompt)

  # send to OpenAI
  res = chat(messages)

  # add latest AI response to messages
  messages.append(res)

  print(res.content)

def converse_without_source_knowledge(query):
  augmented_prompt = query

  # create a new user prompt
  prompt = HumanMessage(
      content=augmented_prompt
  )
  # add to messages
  messages.append(prompt)

  # send to OpenAI
  res = chat(messages)

  # add latest AI response to messages
  messages.append(res)

  print(res.content)

In [181]:
converse_using_source_knowledge("Can you tell me about the code in this python file?")

The code is defining a function called "launch" which imports argparse and sets several parameters including the number of epochs, batch size, image size, number of classes, dataset path, device, learning rate and save path. The function then calls another function called "train" with the specified parameters. 

The code also defines a function called "forward" which takes in three parameters: x, t, and y. It then performs several operations on these parameters using various layers of a UNet model and returns an output.

The code imports various libraries including os, torch, torchvision, PIL, and matplotlib. It also loads a saved checkpoint of a trained UNet model and uses it to generate images using diffusion sampling.


In [182]:
converse_using_source_knowledge("Which dataset is used in this code?")

The dataset used in this code is located at "/content/drive/MyDrive/cifar-64/train". However, the exact name of the dataset is not specified in the provided code.


In [183]:
converse_using_source_knowledge("Does this code run on CPU?")

No, this code is designed to run on the GPU using "cuda".


In [184]:
converse_without_source_knowledge("Summarize our conversation")

Sure! 

In our conversation, you asked me several questions about a provided code, which included functions for launching a model, training it on a dataset, generating images using diffusion sampling, and defining a forward function for the model. 

You asked about the dataset used in the code, but it was not clearly specified in the provided code. You also asked whether the code runs on CPU or GPU, and I confirmed that it is designed to run on the GPU using "cuda". 

I hope my answers were helpful to you!


In [185]:
for message in messages:
  print("**************************")
  print(message)

**************************
content='You are a helpful assistant who helps understand code.'
**************************
content='Great! How can I assist you today?'
**************************
content='Using the contexts below, answer the query.\n\n  Contexts:\n  def launch():\n    import argparse\n    parser = argparse.ArgumentParser()\n    run_name = "DDPM_conditional"\n    epochs = 1\n    batch_size = 4\n    image_size = 64\n    num_classes = 10\n    dataset_path = "/content/drive/MyDrive/cifar-64/train"\n    device = "cuda"\n    lr = 3e-4\n    save_path = "/content/drive/MyDrive/ddpm_conditional"\n    train(run_name, device, num_classes, lr, image_size, batch_size, dataset_path, epochs, save_path)\n\nlaunch()\ndef forward(self, x, t, y):\n        t = t.unsqueeze(-1).type(torch.float)\n        t = self.pos_encoding(t, self.time_dim)\n\n        if y is not None:\n            t += self.label_emb(y)\n\n        x1 = self.inc(x)\n        x2 = self.down1(x1, t)\n        x2 = self.sa1(x2)\n 

## Part B: Write a chatbot prompt to iteratively create a sequence of chats on one particular custom data.

The chatbot should be able to answer the questions based on the text data or multiple documents.

The chatbot should save the conversation in the memory.

Summarize the chats at the end of the conversation.

In [63]:
from datasets import load_dataset

dataset = load_dataset(
    "RaiBP/openwebtext2-first-30-chunks-translation-examples",
    split="train"
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/6521 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/RaiBP___parquet/RaiBP--openwebtext2-first-30-chunks-translation-examples-fd5d48c8a573b892/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [64]:
dataset[0]

{'text': 'Today was the opening of an info centre in the jungle – a collaboration between CMS, help Calais and the Hummingbird Project.\n\nThe space has been set up to share information on things from asylum in the UK and elsewhere, to goings on in the jungle, to information on the history of struggle in Calais, to issues that go beyond Calais that affect travellers (such as deportations and mass arrests). It’s also a space where people living in the jungle and other groups can share information. On the weekends the space will also be used as a clinic. Already in day one there were numerous people dropping in.\n\n[Its worth saying that this is not a space where people can give advice. We just share information.\n\nAlso, If you want to make an info-based project in the jungle, this might be a space you can use].\n\n—————————————————————————————\n\nAujourd’hui était le jour d’ouverture du centre d’information à la Jungle- une collaboration entre CMS et help Calais et Hummingbird Project.

In [73]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY")
# configure client
pc = Pinecone(api_key=api_key)

In [74]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

In [77]:
import time

index_name = 'openwebtext2-rag'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of ada 002
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [78]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [81]:
from tqdm.auto import tqdm  # for progress bar

data = dataset.to_pandas()  # this makes it easier to iterate over the dataset

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)
    # get batch of data
    batch = data.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"{x['id']}" for i, x in batch.iterrows()]
    # get text to embed
    texts = [x['text'] for _, x in batch.iterrows()]
    # embed text
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['text']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

  0%|          | 0/66 [00:00<?, ?it/s]

In [82]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 6521}},
 'total_vector_count': 6521}

In [84]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [104]:
query = "What does the Global Pact for Refugees approved by UN contain?"

vectorstore.similarity_search(query, k=3)

[Document(page_content='United Nations (@TRT)\n\nOn December 17th, the General Assembly of the United Nations (UN) approved the Global Pact for Refugees. This victory for human rights and international cooperation comes one week after the General Assembly approved the Global Pact on Migration.\n\n181 countries signed the pact that outlines steps that the international community will take to guarantee the welfare of refugees. Three countries abstained, and the United States and Hungary voted against it. The United States had previously stated that, although it agreed with most of the Covenant, it did not want to accept a limitation on the detention of asylum seekers.\n\nThe Pact will try to give more support to the countries that host refugees, and when possible, help refugees to return to their countries of origin. It also calls for promoting the independence of refugees. To improve infrastructure and services to refugees, the pact will try to encourage more investment from governments

In [105]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [106]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    United Nations (@TRT)

On December 17th, the General Assembly of the United Nations (UN) approved the Global Pact for Refugees. This victory for human rights and international cooperation comes one week after the General Assembly approved the Global Pact on Migration.

181 countries signed the pact that outlines steps that the international community will take to guarantee the welfare of refugees. Three countries abstained, and the United States and Hungary voted against it. The United States had previously stated that, although it agreed with most of the Covenant, it did not want to accept a limitation on the detention of asylum seekers.

The Pact will try to give more support to the countries that host refugees, and when possible, help refugees to return to their countries of origin. It also calls for promoting the independence of refugees. To improve infrastructure and services to refugees, the pact will try to encourage

In [111]:
messages = [
    SystemMessage(content="You are a helpful assistant who helps text from custom dataset.")
]

In [112]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)
messages.append(res)

The Global Pact for Refugees approved by the United Nations outlines steps that the international community will take to guarantee the welfare of refugees. It aims to give more support to the countries that host refugees, and when possible, help refugees to return to their countries of origin. It also calls for promoting the independence of refugees and encouraging more investment from governments and private sources to improve infrastructure and services to refugees.


In [113]:
prompt = HumanMessage(
    content="When was the Global Pact for Refugees pact signed and who signed it?"
)
messages.append(prompt)
res = chat(messages)
print(res.content)
messages.append(res)

The Global Pact for Refugees was approved by the General Assembly of the United Nations on December 17, 2018. A total of 181 countries signed the pact, while three countries abstained, and the United States and Hungary voted against it.


In [114]:
prompt = HumanMessage(
    content="Summarize our conversation"
)
messages.append(prompt)
res = chat(messages)
messages.append(res)
print(res.content)

In our conversation, I answered your questions regarding the Global Pact for Refugees approved by the United Nations. I provided information about what the pact contains and when it was signed, as well as who signed it. Additionally, I provided some related information about how some migrants sustain themselves during their journeys.


In [116]:
for message in messages:
  print("**************************")
  print(message)

**************************
content='You are a helpful assistant who helps text from custom dataset.'
**************************
content='Using the contexts below, answer the query.\n\n    Contexts:\n    United Nations (@TRT)\n\nOn December 17th, the General Assembly of the United Nations (UN) approved the Global Pact for Refugees. This victory for human rights and international cooperation comes one week after the General Assembly approved the Global Pact on Migration.\n\n181 countries signed the pact that outlines steps that the international community will take to guarantee the welfare of refugees. Three countries abstained, and the United States and Hungary voted against it. The United States had previously stated that, although it agreed with most of the Covenant, it did not want to accept a limitation on the detention of asylum seekers.\n\nThe Pact will try to give more support to the countries that host refugees, and when possible, help refugees to return to their countries of or

In [117]:
pc.delete_index(index_name)

---